<a href="https://colab.research.google.com/github/Satwikram/Tensorflow-Serving/blob/master/TensorFlow%20Serving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stage 1: Install dependencies and setting up GPU environment

In [2]:
!echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  25815      0 --:--:-- --:--:-- --:--:-- 25815
OK


In [3]:
!pip install tensorflow==1.13.1

     |████████████████████████████████| 92.5MB 56kB/s 
     |████████████████████████████████| 3.2MB 22.5MB/s 
     |████████████████████████████████| 368kB 53.7MB/s 
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [4]:
!apt-get update & apt-get install tensorflow-model-server

Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Get:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:12 http://storage.googleapis.com

In [5]:
!pip install requests

## Stage 2: Import project dependencies

In [6]:
import os
import random
import json
import tensorflow as tf
tf.enable_eager_execution()
import requests
import numpy as np
import matplotlib.pyplot as plt
import subprocess
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [7]:
from tensorflow.keras.datasets import cifar10

In [8]:
tf.__version__

'1.13.1'

## Stage 3: Dataset preprocessing

### Loading the dataset

In [9]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [10]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

### Image normalization

In [11]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [12]:
X_train.shape

(50000, 32, 32, 3)

## Stage 4: Defining the model


In [13]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[32, 32, 3]))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


### Compiling the model

In [14]:
model.compile(optimizer='Adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['sparse_categorical_accuracy'])

### Training the model

In [23]:
model.fit(X_train, 
          y_train, 
          batch_size=256, 
          epochs=5)

Epoch 1/10
50000/50000 [==============================] - 266s 5ms/sample - loss: 0.4091 - sparse_categorical_accuracy: 0.8587
Epoch 2/10
50000/50000 [==============================] - 267s 5ms/sample - loss: 0.3522 - sparse_categorical_accuracy: 0.8777
Epoch 3/10
50000/50000 [==============================] - 266s 5ms/sample - loss: 0.3080 - sparse_categorical_accuracy: 0.8922
Epoch 4/10
50000/50000 [==============================] - 267s 5ms/sample - loss: 0.2633 - sparse_categorical_accuracy: 0.9089
Epoch 5/10
50000/50000 [==============================] - 266s 5ms/sample - loss: 0.2155 - sparse_categorical_accuracy: 0.9241
Epoch 6/10
50000/50000 [==============================] - 267s 5ms/sample - loss: 0.1875 - sparse_categorical_accuracy: 0.9360
Epoch 7/10
50000/50000 [==============================] - 266s 5ms/sample - loss: 0.1428 - sparse_categorical_accuracy: 0.9517
Epoch 8/10
50000/50000 [==============================] - 266s 5ms/sample - loss: 0.1175 - sparse_categorical_a

### Model evaluation

In [24]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 17s 2ms/sample - loss: 1.5171 - sparse_categorical_accuracy: 0.7248


In [25]:
print("Test accuracy is {}".format(test_accuracy))

Test accuracy is 0.7247999906539917


## Stage 5: Saving the model for production

### Creating the directory for the model

In [26]:
MODEL_DIR = "model/"
version = 1

In [27]:
export_path = os.path.join(MODEL_DIR, str(version))

In [28]:
export_path

'model/1'

In [29]:
if os.path.isdir(export_path):
  !rm -r {export_path}

### Saving the model for the TensorFlow Serving

In [ ]:
tf.saved_model.simple_save(tf.keras.backend.get_session(), export_dir = export_path, 
                           inputs = {"input_image":model.input}, outputs = {t.name: t for t in model.output})

## Stage 6: Setting up the production environment

### Exporting the MODEL_DIR to environment variables

In [ ]:
os.environ['MODEL_DIR'] = os.path.abspath(MODEL_DIR)

### Running the TensorFlow Serving REST API

In [ ]:
%%bash --bg
nohup tensorflow_model_server --rest_api_port=8501 --model_name=cifar10 --model_base_path="${MODEL_DIR}" >server.log 2>&1

In [ ]:
!tail server.log

## Stage 7: Creating the first POST request

In [ ]:
random_image = np.random.randint(0, len(X_test))
random_image

### Creating the JSON data object

In [ ]:
data = json.dumps({"signature_name":"serving_default", "instances":[X_test[random_image].tolist()]})

In [ ]:
data